In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

As you see , this data set is about the water quality and At its most basic level, potabible water relates to the safety of water.

Many questions begin to emerge. we want to find out the Water quality and Potability of it 

* Are we able to consume all fresh water types?

* What percentage of the worlds fresh water can be accessed?

* Has the water table increased as sea levels have rised?


1. ph: pH of 1. water (0 to 14).
2. Hardness: Capacity of water to precipitate soap in mg/L.
3. Solids: Total dissolved solids in ppm.
4. Chloramines: Amount of Chloramines in ppm.
5. Sulfate: Amount of Sulfates dissolved in mg/L.
6. Conductivity: Electrical conductivity of water in μS/cm.
7. Organic_carbon: Amount of organic carbon in ppm.
8. Trihalomethanes: Amount of Trihalomethanes in μg/L.
9. Turbidity: Measure of light emiting property of water in NTU.
10. Potability: Indicates if water is safe for human consumption. Potable -1 and Not potable -0

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [ ]:
df=pd.read_csv('../input/water-potability/water_potability.csv')
df.head(5)

In [ ]:
df.info()

In [ ]:
# Number of rows and columns
df.shape

In [ ]:
# we want to divide the data into categorical and numerical 
cat_cols=df.select_dtypes(include='object')
num_cols=df.select_dtypes(exclude='object')

In [ ]:
# As you see, we do not have any categorical attributes
for cat in cat_cols.columns:
    print(f"{cat}")
    print(cat_cols[cat].unique())
    print(cat)

In [ ]:
# we find out all information about the data
num_cols.describe()

In [ ]:
# sum of the numbers and divide to number by amount of the numbers
df.mean()

In [ ]:
#Middle of the point
df.median()

In [ ]:
# we want to divide our goal into target and non_target
target=df[df['Potability']==1]
non_target=df[df['Potability']==0]

In [ ]:
target_cols=['Potability']
cat_cols=df.nunique()[df.nunique()<8].keys().tolist()
cat_cols=[x for x in cat_cols if x not in target_cols]
num_col=[x for x in df.columns if x not in cat_cols+ target_cols]

In [ ]:
median=df['ph'].median()
df['ph'].fillna(median,inplace=True)
median=df['Sulfate'].median()
df['Sulfate'].fillna(median,inplace=True)
median=df['Trihalomethanes'].median()
df['Trihalomethanes'].fillna(median,inplace=True)

In [ ]:
# Display missing values using a heatmap to understand if any patterns are present( we cleaned all the missing values by median and use heatmap for better understanding)
plt.figure(figsize=(15,8))
sns.heatmap(df.isnull());

In [ ]:
# As you see, the most portion is 0, it means it is not potable(Almost 60%)
df['Potability'].value_counts(normalize=True)*100

In [ ]:
# we use filter to find out the the potability is 0 , sulfate is more than 300 , chloramines more than 10 and hardness of the water is more than 220
df[(df['Potability']==0) &(df['Sulfate']>300)&(df['Chloramines']>10)&(df['Hardness']>220)]

In [ ]:
# we want to use groupby with sum and counts and also with potability

# Read the data
df_grouped_sum=df.groupby('Potability',as_index=False)['Conductivity'].agg('sum').rename(columns={'Conductivity':'Conductivity_Sum'})
df_grouped_cnt=df.groupby('Potability',as_index=False)['Conductivity'].agg('count').rename(columns={'Conductivity':'Conductivity_Count'})
#Merge the data
df_grouped_Avg=df_grouped_sum.merge(df_grouped_cnt,left_on='Potability',right_on='Potability',how='inner')

df_grouped_Avg.loc[:,'Average of Conductivity']=df_grouped_Avg['Conductivity_Sum']/df_grouped_Avg['Conductivity_Count']


# The final result
df_grouped_Avg.sort_values('Conductivity_Sum',ascending=False)

In [ ]:
# we want to use groupby, sum and count with Potability by Hardness
# Read the data
df_grouped_sum=df.groupby('Potability',as_index=False)['Hardness'].agg('sum').rename(columns={'Hardness':'Hardness_Sum'})
df_grouped_cnt =df.groupby('Potability',as_index=False)['Hardness'].agg('sum').rename(columns={'Hardness':'Hardness_Count'})
# Merge the 2 lines of code 
df_grouped_Avg=df_grouped_sum.merge(df_grouped_cnt,left_on='Potability',right_on='Potability',how='inner')

df_grouped_Avg.loc[:,'Average of Hardness']=df_grouped_Avg['Hardness_Sum']/df_grouped_Avg['Hardness_Count']

# Final result
df_grouped_Avg.sort_values('Hardness_Sum',ascending=False)

# We want to use some visualization parts with different attributes

In [ ]:
plt.rcParams['figure.figsize']=(10,9)
df['Potability'].value_counts().sort_values(ascending=False).plot.bar(color='orange')
plt.xlabel('amount of the potability')
plt.ylabel('count')
plt.show()
# As you see , most portion of water is potable( zero is more than one)

In [ ]:
# As you see, the range of the ph for non potable water is between 6 to 8 and for potable water is 6 to 7.5
sns.violinplot(y='ph',x='Potability',data=df, color='aqua')

In [ ]:
plt.figure(figsize=(10,6))
sns.relplot('Conductivity','Turbidity',data=df, color='pink')
plt.title('Show the amount of the conductivity with the Turbidity')
plt.show()
# As you see, the most amount of the conductivity is between 400 to 550 and the amount for Turbidity is between 3 to 5

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot('Chloramines','Trihalomethanes',data=df,color='yellow')
plt.show()
# As you see, we use scatterplot to visualize the amount of the Chloramines which the most portion is between 5 to 9 and the amount of the Trihalomethanes is 50 to 100 μg/L

In [ ]:
# Use the distplot for ph with mean and median
sns.distplot(df['ph'],kde=False)
plt.axvline(x=df.ph.mean(),linewidth=3, color='r', label="mean", alpha=0.5)
plt.axvline(x=df.ph.median(),linewidth=3,color='y',label='median',alpha=0.5)

# set the labels
plt.xlabel('ph')
plt.ylabel('count')
plt.title('Distribution of ph')
plt.legend(['mean','median'])

print(f'Mean pH value {df.ph.mean()} \n Median pH value {df.ph.median()} \n Min pH value {df.ph.min()} \n Max pH value {df.ph.max()}')

In [ ]:
sns.distplot(df['Hardness'],kde=False)
plt.axvline(x=df.Hardness.mean(),linewidth=3,color='g',label='Mean',alpha=0.4)
plt.axvline(x=df.Hardness.median(),linewidth=3,color='purple',label='Median',alpha=0.5)

# set the labels
plt.xlabel('Hardness')
plt.ylabel('count')
plt.title('Distribution of Hardness',size=10)
plt.legend(['Mean','Median'])

# result with percentage
print(f'Mean Hardness value {df.Hardness.mean()} \n Median Hardness value {df.Hardness.median()} \n Min Hardness value {df.Hardness.min()} \n Max Hardness value {df.Hardness.max()}')

In [ ]:
sns.histplot(df['Sulfate'],kde=True,color='silver')
plt.show()
# In this graph, the most portion of sulfate is 330 mg/L and the amount of this is more than 900

In [ ]:
sns.distplot(df['Organic_carbon'],kde=False)
plt.axvline(x=df.Organic_carbon.mean(),linewidth=3,color='y',label='Mean',alpha=0.4)
plt.axvline(x=df.Organic_carbon.median(),linewidth=3,color='r',label='Median',alpha=0.4)

# set the labels
plt.xlabel('Organic_carbon')
plt.ylabel('count')
plt.title('Distribute of Organic_carbon',size=12)
plt.legend(['Mean','Median'])

print(f'Mean Organic_carbon value {df.Organic_carbon.mean()} \n Median Organic_carbon value {df.Organic_carbon.median()} \n Min Organic_carbon value {df.Organic_carbon.min()} \n Max Organic_carbon value {df.Organic_carbon.max()}')

In [ ]:
# Preprocessing
from sklearn.preprocessing import scale
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 

# performance metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

In [ ]:
X = df.drop(['Potability','ph'],axis=1)
y = df['Potability'].values

In [ ]:
X.head(5)

In [ ]:
# k-NN classifier

# Split into training and test set
X_train , X_test , y_train , y_test=train_test_split(X , y , test_size=0.3 , random_state=1234, stratify= y )

# Create a k-NN classifier with 7 neighbors
knn = KNeighborsClassifier(n_neighbors=7)

# Fit the classifier to the training data
knn.fit(X_train, y_train)

# Print the accuracy
print(knn.score(X_test, y_test))

In [ ]:
# we want to know the performance of the KNeighbors for testing and training result
neighbors = np.arange(1, 12)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

for i, k in enumerate(neighbors):
    # Train and test the model
    knn = KNeighborsClassifier(n_neighbors=k)

    # train the model
    knn.fit(X_train, y_train)
         
    # compute the accuracy
    train_accuracy[i] = knn.score(X_train, y_train)
        
    # compute the test
    test_accuracy[i]=knn.score(X_test, y_test)
    
        
# result
plt.title('kNeighbors: Varying Number of Neighbors')
plt.plot(neighbors , test_accuracy , label='Test_Acuracy')
plt.plot(neighbors, train_accuracy , label='Train_Accuracy')
plt.legend()
plt.xlabel('number of neighbors')
plt.ylabel('Accuracy')
plt.show()
# You can see the differences the number of the neighbors and also train and test of the KNeighbors and amount of the acuracy

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape,y_train.shape)

In [ ]:
models=[]
models.append(('LR ',LogisticRegression(random_state=12345)))
models.append(('DT',DecisionTreeClassifier(random_state=12345)))
models.append(('RF',RandomForestClassifier(random_state=12345)))
models.append(('NN',MLPClassifier(random_state=12345)))
models.append(('KN', KNeighborsClassifier()))
models.append(('SVM',SVC(random_state=12345)))

#value
result=[]
name=[]

# We want to find out what is the classification report and confusion matrix and when we use it:
* Percision: measure the model performance on measuring the count of true positives in correct manner out of all positive predictions made.

* Recall score :is used to measure the model performance in terms of measuring the count of true positives in correct manner out of all the actual positive values.

* F1-score :is harmonic mean of precision and recall score and is used as a metrics in the scenarios where choosing either of precision or recall score can result in compromise in terms of model giving high false positives and false negatives respectively.

*  Macro average gives each prediction similar weight while calculating loss but there might be case when your data might be imbalanced and you want to give importance to some prediction more (based on their proportion), there you use 'weighted' averag


In [ ]:
# As you see, these are the result of making model( Random Forest has the highest percent among another )
for name , model in models:
    model.fit(X_train, y_train)
    predictions=model.predict(X_test)
    accuracy=accuracy_score(y_test, predictions)
    msg = "%s: (%f)" % (name, accuracy)
    print(msg)
    print(classification_report(y_test, predictions))
    print(confusion_matrix(y_test, predictions))
    